<a href="https://colab.research.google.com/github/saieshtadepalli06/Capstone-Initiation/blob/main/de_lab_1_saieshwar_tadepalli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Engineering - Lab 1


## Intro to table of contents

In this lab we'll be introduced to the Google Colab environment. To start this notebook simply go to "Runtime" and choose "Run All". This will connect Colab to your virtual machine and run the code needed to get your database setup.

You'll notice that notebooks are setup with a table of contents navigation bar on the left nav bar of your screen. Mouse over and hover to see which options are available and click "Table of Contents" to see the major sections to this Notebook.  

## Database setups (do not modify)

In [2]:
# setups including getting the database

!pip -q install --upgrade ipython
!pip -q install --upgrade ipython-sql

!wget -O northwind.db https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db

import sqlite3
con = sqlite3.connect("northwind.db")

%load_ext sql
%sql sqlite:///northwind.db

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.2/809.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.20.0 which is incompatible.
--2024-01-25 04:43:40--  https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db [following]
--2024-01-25 04:43:40--  https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.19

## ERD of the Northwind Database


![ERD](https://github.com/matthewpecsok/data_engineering/blob/main/data/Northwind_ERD.png?raw=true)


Using %sql allows for one line sql statements
Using %%sql allows for multi-line sql statements and better readability for long sql queries.

In [3]:
%sql select * from products limit 4

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22,53,0,0,0


In [4]:
%%sql

select
count(1) as product_count
from
products

,product_count
0,77


## Q1: How many products have a UnitPrice of greater than 18?

In [6]:
%%sql

select count(1) as product_count
from products
where UnitPrice > 18

,product_count
0,43


## Q2: What are the top 5 customers based on order quantity?


In [7]:
%%sql

select Customers.CustomerID as Customer_ID, Customers.ContactName as Contact_Name, sum('Order Details'.Quantity) as Total_Order_Quantity
from Customers
left join Orders
on Customers.CustomerID = Orders.CustomerID
left join 'Order Details'
on Orders.OrderID = 'Order Details'.OrderID
group by Customer_ID, Contact_Name
order by Total_Order_Quantity desc
limit 5

,Customer_ID,Contact_Name,Total_Order_Quantity
0,SAVEA,Jose Pavarotti,4958
1,ERNSH,Roland Mendel,4543
2,QUICK,Horst Kloss,3961
3,HUNGO,Patricia McKenna,1684
4,FRANK,Peter Franken,1525


## Q3: Create a dataset showing the OrderId and CategoryName for all orders in the database. Limit the result set to 5 rows. Order by CategoryId.



In [8]:
%%sql

select Orders.OrderID as Order_ID, Categories.CategoryName as Category_Name
from Orders
left join 'Order Details' on Orders.OrderID = 'Order Details'.OrderID
left join Products on Products.ProductID = 'Order Details'.ProductID
left join Categories on Categories.CategoryID = Products.CategoryID
order by Categories.CategoryID
limit 5

,Order_ID,Category_Name
0,10253,Beverages
1,10254,Beverages
2,10255,Beverages
3,10257,Beverages
4,10258,Beverages


## Q4: Show the counts of all orders in the database grouped by ShipRegion, Sort by the count in descending order. The columns should be named RegionCount and ShipRegion respectively. Filter this dataset to include only counts greater than 100.



In [9]:
%%sql

select count(*) AS RegionCount, ShipRegion
from Orders
where ShipRegion is not null
group by ShipRegion
having RegionCount > 100
order by RegionCount desc


,RegionCount,ShipRegion
0,276,Western Europe
1,152,North America
2,145,South America


## Q5: Create a single query showing the max, min and average unit price from the products table. Round the average price to 2 decimal places. In the query, rename the average column name to average_unit_price

In [10]:
%%sql

select max(UnitPrice) AS max_unit_price,
    min(UnitPrice) AS min_unit_price,
    round(avg(UnitPrice), 2) AS average_unit_price
from Products

,max_unit_price,min_unit_price,average_unit_price
0,263.5,2.5,28.87


## Q6: Create a query returning 4 columns 1) the original_total column which is the unitprice*quantity from the order_detail table 2) the discounted_price column which is the similar to original_total except that it shows reduces the total by the discount amount from the order_detail table 3) the discount 4) the company name Finally: filter the dataset for CompanyName Ernst Handel and original_total > 4000

In [12]:
%%sql

select
    ('Order Details'.UnitPrice * 'Order Details'.Quantity) AS original_total,
    (('Order Details'.UnitPrice - ('Order Details'.UnitPrice * 'Order Details'.Discount)) * 'Order Details'.Quantity) AS discounted_price,
    'Order Details'.Discount,
    Customers.CompanyName
from 'Order Details'
left join Orders ON 'Order Details'.OrderID = Orders.OrderID
left join Customers ON Orders.CustomerID = Customers.CustomerID
where Customers.CompanyName = 'Ernst Handel' and ('Order Details'.UnitPrice * 'Order Details'.Quantity) > 4000

,original_total,discounted_price,Discount,CompanyName
0,4216.0,4005.2,0.05,Ernst Handel
1,6360.0,6042.0,0.05,Ernst Handel
2,6050.0,6050.0,0.00,Ernst Handel
3,4322.5,4322.5,0.00,Ernst Handel


## Q7: Create a query returning the orderdate formatted as a year named order_year, the count of orders named order_count grouped by year and ordered by order_year. This query should return 3 rows 1 for each year with the count for each year respectively.

In [13]:
%%sql

select
    strftime('%Y', Orders.OrderDate) AS order_year,
    count(*) AS order_count
from Orders
group by strftime('%Y', Orders.OrderDate)
order by order_year;

,order_year,order_count
0,2016,152
1,2017,408
2,2018,270


## Q8: Based on a criteria of your choosing: Who are the top 5 Employees in the database? Create a query to answer this question. You must join to orders at the least. No, you can't just query the employees table and return the highest employeeids, nice try though!

In [15]:
%%sql

select
    Employees.EmployeeID,
    Employees.FirstName || ' ' || Employees.LastName as EmployeeName,
    SUM('Order Details'.UnitPrice * 'Order Details'.Quantity * (1 - 'Order Details'.Discount)) as TotalSales
from Employees
left join Orders on Employees.EmployeeID = Orders.EmployeeID
left join 'Order Details' on Orders.OrderID = 'Order Details'.OrderID
group by Employees.EmployeeID, EmployeeName
order by TotalSales desc
limit 5

,EmployeeID,EmployeeName,TotalSales
0,4,Margaret Peacock,232890.8460
1,3,Janet Leverling,202812.8430
2,1,Nancy Davolio,192107.6045
3,2,Andrew Fuller,166537.7550
4,8,Laura Callahan,126862.2775


## Q9: Using the sqlite_master table to query, how many tables are there in the database?


In [16]:
%%sql

select count(*) as table_count
from sqlite_master
where type = 'table'

,table_count
0,14


## Q10: Are there any views in the database? How many?

In [17]:
%%sql

select
    count(*) as view_count
from sqlite_master
where type = 'view'

,view_count
0,17


## Q11: Create a new table in the database. Give it a name and create 3 columns of your choosing. Write a query to return the count of rows in the table.

In [ ]:
%sql --your code here

## Q12: Insert 5 rows into your new table.

In [ ]:
%sql --your code here

## Q13: Select all the rows and columns from your new table.

In [ ]:
%sql --your code here

## Q14: Join the customer table to the orders table with a left join, get the count of the rows.


In [18]:
%%sql

select count(*) AS count
from Customers
left join Orders on Customers.CustomerID = Orders.CustomerID

,count
0,834


## Q15: Join the customer table to the orders table with a inner join, get the count of the rows. in a text block explain why the count is different.

In [19]:
%%sql

select count(*) AS count
from Customers
inner join Orders on Customers.CustomerID = Orders.CustomerID

,count
0,830


add additional sections as needed to complete the lab

In [ ]:
# replace ###### with your file name
# make sure you have your google drive mounted.

!cp "/content/drive/MyDrive/Colab Notebooks/######.ipynb" ./
!jupyter nbconvert --to html "######.ipynb"